In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

In [6]:
path = r"C:\Users\shahe\OneDrive\Desktop\NLP Sessions\Project\Diagnosify\04012024HR8220R.pdf"

In [7]:
loader = PyMuPDFLoader(path)

In [8]:
pages = loader.load_and_split()

with open('test_file.txt', 'w') as f:
    f.write(pages[0].page_content)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
model_name = "ruslanmv/Medical-Llama3-8B"
device_map = 'auto'
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16,)
model = AutoModelForCausalLM.from_pretrained( model_name,quantization_config=bnb_config, trust_remote_code=True,use_cache=False,device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def askme(question):
    sys_message = ''' 
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''   
    # Create messages structured for the chat template
    messages = [{"role": "system", "content": sys_message}, {"role": "user", "content": question}]

    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    
    # Extract and return the generated text, removing the prompt
    response_text = tokenizer.batch_decode(outputs)[0].strip()
    answer = response_text.split('<|im_start|>assistant')[-1].strip()
    return answer
# Example usage
# - Context: First describe your problem.
# - Question: Then make the question.

question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, 
increased sensitivity to cold, and dry, itchy skin. 
Could these symptoms be related to hypothyroidism? 
If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

print(askme(question))

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\shahe\AppData\Roaming\Python\Python312\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [1]:
pip install --quiet transformers torch

Note: you may need to restart the kernel to use updated packages.


In [18]:
llm = ChatOllama(
    model_name=model_name,
    tempreture=0
)

In [26]:
messages = [
    (
        "system",
        "You are a helpful AI doctor, who can help diagnose medical conditions and give them recommendations for treating the disease. If you don't know the answer to a specific medical inquiry, advise seeking professional help.",
    ),
    (
        "human", "I am a 20 year old male, I have been having pain in my lower left and right side of my body. I have also been experiencing nausea and vomiting. What could be the cause of this?"
    ),
]

ai_msg = llm.invoke(messages)

In [27]:
print(ai_msg.content)


Thank you for reaching out to me! I'm here to help you. The symptoms you've described are quite non-specific, but there are several possible causes that could be contributing to your discomfort. As a responsible AI doctor, I must advise you to consult with a qualified medical professional for an accurate diagnosis and appropriate treatment.

However, based on the information you've provided, here are some potential causes of your symptoms:

1. Gastrointestinal issues: The pain in your lower left and right side could be related to a gastrointestinal issue such as irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), or a food poisoning. The nausea and vomiting may also be due to a gastrointestinal problem.
2. Appendicitis: Although rare, it's possible that you might have appendicitis, which can cause pain in the lower right abdomen. If left untreated, it can lead to serious complications.
3. Kidney stones or infections: Kidney stones or urinary tract infections can cause fl